ANALYTICAL DECOMPOSITION OF MPS AS 1/2-QUBIT GATES
Input:
    -N = sites
Items:
    -mps = initial state with 2^N entries
    -psi = initial state as rank-N tensor (smallest case is rank-3 with shapes: (2,2,2))
    -B_sites = list of single rank-3 tensors
    

In [1]:
import numpy as np
from numpy import linalg as LA
from ncon import ncon
from scipy.linalg import null_space as NS

In [2]:
N = 3
mps = np.random.random(2**N) #+ np.random.random(2**N) * 1j
mps_norm = mps / LA.norm(mps)
psi_norm = mps_norm / LA.norm(mps_norm)
psi = psi_norm.reshape(2,2,2)
print(LA.norm(psi))


1.0


In [3]:
B_sites = [0+0j for _ in range(0,N)]

U,S,Vh = LA.svd(psi.reshape(4,2), full_matrices=False)

B_sites[2] = Vh.reshape(2,2)
temp = ((U[:,:2]@np.diag(S[:2])).reshape(2,-1))/LA.norm(S)
U,S,Vh = LA.svd(temp.reshape(2,4), full_matrices=False)
B_sites[1] = Vh.reshape(2,2,2)
temp = ((U[:,:2]@np.diag(S[:2])).reshape(2,-1))/LA.norm(S)
B_sites[0] = temp

CHECK CONDIZIONI RIGHT-CANONICAL FORM


In [4]:
print(np.round(ncon([B_sites[2],B_sites[2].conjugate()],[[-1,1],[-2,1]]))) #A_N@A_N* = I_2
print(np.round(ncon([B_sites[1],B_sites[1].conjugate()],[[-1,1,2],[-2,1,2]]))) #A_n@A_n* = I_2
print(np.round(ncon([B_sites[0],B_sites[0].conjugate()],[[1,2],[1,2]]))) #A_1@A_1 = 1


[[1. 0.]
 [0. 1.]]
[[1. 0.]
 [0. 1.]]
1.0


CALCOLO NULL SPACE DEI VARI TENSORI

In [8]:
null_K = [0+0j for _ in range(0,N)] 
print("Tensor 3 does not need addition because is a 2x2 matrix")
print(B_sites[2])
null_K[2] = NS(B_sites[2])
print(null_K[2])

print("\nTensor 2 needs 2 2x2-vectors from the null space")
#print(B_sites[1].reshape(4,2))
null_K[1] = NS(B_sites[1].reshape(2,4).T)
#print("First: ",null_K[1][:,0].reshape(4,1))
#print("Second: ",null_K[1][:,1].reshape(4,1))
#print("\nTensor 1 needs 3 2x2-vectors from the null space")
#print(B_sites[0].reshape(4,1))
null_K[0] = NS(B_sites[0].reshape(1,4).T)
#print("First:",null_K[0][:,0].reshape(4,1))
#print("Second: ",null_K[0][:,1].reshape(4,1))
#print("Third: ",null_K[0][:,2].reshape(4,1))

Tensor 3 does not need addition because is a 2x2 matrix
[[-0.90448363 -0.42650835]
 [ 0.42650835 -0.90448363]]
[]

Tensor 2 needs 2 2x2-vectors from the null space
[[ 0.71681112 -0.20857697]
 [ 0.61727668  0.24828807]
 [ 0.60542202 -0.25886295]
 [-0.74350007 -0.11688369]]


CREAZIONE DELLE MATRICI G

In [6]:
G = [0+0j for _ in range(N)]
G[0] = np.block([[B_sites[0].reshape(4,1).T], [null_K[0].T]])
G[1] = np.block([[B_sites[1].reshape(4,2).T], [null_K[1].T]])
G[2] = B_sites[2]

for g in G:
    print(ncon([g,g.T],[[-1,1],[-2,1]]))


[[ 1.00000000e+00  3.51163064e-17  1.46184155e-16 -8.19628432e-17]
 [ 3.17414888e-17  1.00000000e+00  1.27899812e-17 -4.46505309e-18]
 [ 1.32161480e-16  9.38492864e-18  1.00000000e+00 -1.66848254e-17]
 [-8.00431469e-17 -4.46505309e-18 -3.04026385e-17  1.00000000e+00]]
[[ 0.3398952   0.17130446  0.51473961 -1.42928337]
 [-0.10379043  0.19376037 -0.19321161  0.07031069]
 [-0.0150881  -0.47984308  0.12175976  0.10825404]
 [-0.19155941 -0.11452521 -0.30428609  1.08431752]]
[[ 0.63618126  0.77153963]
 [-0.77153963  0.63618126]]


CHECK UNITARIETA'

In [206]:
G_rank4 = [0+0j for _ in range(N)]
G_rank4[0] = G[0].reshape(2,2,2,2)
G_rank4[1] = G[1].reshape(2,2,2,2)
G_rank4[2] = G[2].reshape(2,2) 